In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("/Users/anuragchaubey/SalesFlux/data/sales_data_500k.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ProductID      500000 non-null  object
 1   Location       500000 non-null  object
 2   Date           500000 non-null  object
 3   UnitsSold      500000 non-null  int64 
 4   PromoCodeUsed  450000 non-null  object
dtypes: int64(1), object(4)
memory usage: 19.1+ MB


In [4]:
df.sample(10)

,ProductID,Location,Date,UnitsSold,PromoCodeUsed
458884,P0494,L01,2024-06-03,10,No
268932,P0400,L03,2024-10-20,0,Yes
151671,P0198,L01,2024-05-04,2,No
291714,P0342,L04,2024-02-08,8,No
390428,P0370,L10,2024-08-09,3,No
168067,P0027,L05,2024-04-01,3,NaN
254643,P0166,L10,2024-03-26,0,Yes
115939,P0165,L03,2024-05-22,9,Yes
157786,P0284,L01,2024-04-24,2,No
359708,P0170,L04,2024-06-13,5,No


In [5]:
df.nunique()

ProductID        500
Location          10
Date             365
UnitsSold         11
PromoCodeUsed      2
dtype: int64

In [6]:
df.duplicated().sum()

np.int64(4194)

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df.isnull().sum()

ProductID            0
Location             0
Date                 0
UnitsSold            0
PromoCodeUsed    49909
dtype: int64

# filling missing data of PromoCodeUsed column

In [10]:
# logic is that, extracting 100 data of PromoCodeUSed for each productId , counting the number of "no" and "yes".. fill the value with most number
sampled_df = df.groupby('ProductID').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)


/var/folders/wz/4tdnltpd0j50hb4mv_kk_0140000gn/T/ipykernel_9486/3544640990.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('ProductID').apply(lambda x: x.sample(min(len(x), 100))).reset_index(drop=True)


In [11]:
promo_counts = sampled_df.groupby(['ProductID', 'PromoCodeUsed']).size().unstack(fill_value=0)


In [12]:
# Create a dictionary with ProductID as key and majority promoCodeUsed as value
promo_majority = {}

for pid in promo_counts.index:
    if promo_counts.loc[pid, 'Yes'] > promo_counts.loc[pid, 'No']:
        promo_majority[pid] = 'Yes'
    else:
        promo_majority[pid] = 'No'


In [13]:
def fill_promo(row):
    if pd.isnull(row['PromoCodeUsed']):
        return promo_majority.get(row['ProductID'], 'No')  # Default 'No' if ProductID not found
    else:
        return row['PromoCodeUsed']

df['PromoCodeUsed'] = df.apply(fill_promo, axis=1)


In [14]:
print("Missing promoCodeUsed after fill:", df['PromoCodeUsed'].isnull().sum())


Missing promoCodeUsed after fill: 0


In [15]:
df.isnull().sum()

ProductID        0
Location         0
Date             0
UnitsSold        0
PromoCodeUsed    0
dtype: int64

In [16]:
df.duplicated().sum()

np.int64(994)

In [17]:
df.drop_duplicates(inplace=True)

In [18]:
df.sample(10)

,ProductID,Location,Date,UnitsSold,PromoCodeUsed
90451,P0439,L07,2024-03-20,5,No
443963,P0428,L09,2024-08-15,1,No
222752,P0308,L05,2024-11-10,10,Yes
5974,P0155,L08,2024-10-31,4,No
412446,P0453,L07,2024-09-27,4,No
285177,P0278,L04,2024-02-01,3,No
473863,P0493,L01,2024-08-05,10,No
100390,P0330,L04,2024-12-13,7,Yes
362518,P0219,L03,2024-08-12,6,Yes
381579,P0030,L04,2024-08-12,7,No


In [19]:
df.dtypes

ProductID        object
Location         object
Date             object
UnitsSold         int64
PromoCodeUsed    object
dtype: object

In [20]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [21]:
df.dtypes

ProductID                object
Location                 object
Date             datetime64[ns]
UnitsSold                 int64
PromoCodeUsed            object
dtype: object

In [22]:
negative_units = df[df['UnitsSold'] < 0]

print(f"Total negative UnitsSold rows: {len(negative_units)}")

Total negative UnitsSold rows: 0


# outliers

In [23]:
Q1 = df['UnitsSold'].quantile(0.25)
Q3 = df['UnitsSold'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Outlier rows
outliers = df[(df['UnitsSold'] < lower_bound) | (df['UnitsSold'] > upper_bound)]

print(f"Total outliers in UnitsSold: {len(outliers)}")


Total outliers in UnitsSold: 0


In [24]:
outliers.sort_values(by='UnitsSold', ascending=False).head(10)


,ProductID,Location,Date,UnitsSold,PromoCodeUsed


In [25]:
outliers['ProductID'].value_counts().head(10)


Series([], Name: count, dtype: int64)

In [26]:
df.describe()

,Date,UnitsSold
count,494812,494812.000000
mean,2024-06-30 23:00:51.888797952,5.001069
min,2024-01-01 00:00:00,0.000000
25%,2024-04-01 00:00:00,2.000000
50%,2024-07-01 00:00:00,5.000000
75%,2024-09-30 00:00:00,8.000000
max,2024-12-30 00:00:00,10.000000
std,NaN,3.162314


In [27]:
df.to_csv("/Users/anuragchaubey/SalesFlux/data/processed/sales_data_cleaned.csv",index=False)